# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** The goal of this project is to identify students who might need early intervention, so we are trying to find a **decision boundary** where to split the students that will fail versus students that will pass to graduate.  The output of this project is **discrete**:  **(class labels)**.  Either the student needs early intervention or not: a **boolean** that is linked a predicted label of **fail** to graduate.  A **regression** supervised learning problem involves continuous outputs: **(numbers)** and we are trying to find a **"best fit line"**, which does not fit the goal of this project.  This is a **classification** supervised learning problem because the output is a **boolean class label (yes|no)**.  Either the student is identified as might needing early intervention or not based on their predicted **fail** class label.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [22]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [95]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
# Ignoring "passed" as a feature, since this will be the target variable.
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = len(student_data[(student_data.passed == 'yes')])

# TODO: Calculate failing students
n_failed = len(student_data[(student_data.passed == 'no')])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)


Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 0.67%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [54]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()


Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [83]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [56]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, train_size=num_train, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

#### Learning Model Selection Method:
We know from *Minimum Description Length* in *Bayesian Methods: Bayesian Learning* and *Preference Bias* in *Artificial Neural Networks: Neural Networks* sections that in general the best supervised learning model has a minimum description length, or a preference bias for the least steps to get to the correct answer: the *Occam's Razor* principle.  This means that we should not unneccessarily choose a more complex model when a simpler model give us the same accuracy.

Following this principle, e.g.: the simplest model is the best, and we should only add additional features, learning models, etc. (complexity) only to gain required additional accuracy, I recommend we start with **Decision Tree Classifier**, and then use **Support Vector Machines Classifier (SVC)**, or **Ensemble Methods: Random Forest Classifier (RandomForestClassifier)** learning models only if the required accuracy justify the additional computational cost.

##### Decision Tree Classifier:
This learning model uses a tree structure to represent decision boundaries, where a series of questions are asked until a final descrete output answer is obtained.  There is an ID3 algorithm that builds the decision tree structure from splitting on the given training dataset.  Afterwards, the training dataset is no longer needed, and the decision tree representation is used instead to predict the target concept, which, in this case, if a student may fail graduation and need early intervention.  A decision tree representation is made up of three components:

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/CART_tree_titanic_survivors.png" title="A Decision Tree showing survival of passengers on the Titanic, CC BY-SA 3.0 by Stephen Milborrow">

- `Decision Nodes:` Where a feature, feature range or a feature vector is selected from the dataset and a question is asked.  This is represented as **`questions..?`** in the diagram above.  The answer values to the question are what the edges (branches) represent in the tree.  The `Decision Tree` starts at the `Root Decision Node`, where hopefully we start with good splits at the top.
- `Edges|Branches:` The descrete value of a feature, feature range or feature vector, represented by encircled values and line segments from a `Decision Node` in the diagram above (e.g.: `yes` or `no`).  `Edges|Branches` connect to childen `Decision Nodes` that picks another feature, feature range or feature vector and ask another question to generate additional `Edges|Branches` below them.  Eventually, the `Edges|Branches` will terminate on `Leafs Nodes`.
- `Leafs Nodes:` These are the final descrete outputs after a series of specific questions and answers paths are traverse.  They are also represented as red or green encircled values in the diagram above.

Some real-world applications for this learning model includes: 
- Predicting Library Book Use
- Exploring the Relationship Between the Research Octane Number and Molecular Substructures
- Characterization of Leiomyomatious Tumors
- Star/Cosmic-Ray Classification in Hubble Space Telescope Images.

Excerpts from: http://booksite.elsevier.com/9780124438804/leondes_expert_vol1_ch3.pdf, section IX. Practical Applications.

The strengths of the Decision Trees model are:
- Best used for classifications.
- Can handle descrete inputs.
- Can handle continuous inputs using ranges as decision boundaries.
- Uses Information Gain formular to decise how to split the decision boundary in the dataset.
- Very easy to understand.  A whitebox model.

Their limits are:
- Performs poorly if ID3 algorithm cannot find a good split near the top.
- Possible exponential node growth as more features are added (n!).
- Possible exponential leaf growth as more nodes are generated (n<sup>2</sup>).
- Prone to overfit when max_depth is set too high.
- ID3 algorithm will continue until all outputs are decided in the training unless we prune.

This learning model follows **Occam's Razor**, it is simple, as long as the Student dataset has a well defined decision boundary that can be split near the top of the decision tree.  While there are many features to evaluate for good splits at the top, the dataset is not large, so training and testing time should be minimal.  We may require to tune the **Decision Tree Classifier** learning model so that overfitting does not lower its prediction accuracy as we add additional training data or features.  Once the decision tree is contructed and optimized, the computational query time should also be minimal.

##### Support Vector Machines Classifier (SVC):

##### Ensemble Methods: Random Forest Classifier (RandomForestClassifier):



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [93]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [96]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier()
clf_B = SVC()
clf_C = RandomForestClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)




Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.6942.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0026 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7597.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0035 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7154.
Training a SVC using a training set size of 100. . .
Trained model in 0.0023 seconds
Made predictions in 0.0018 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0016 seconds.
F1 score for test set: 0.7838.
Training a SVC using a training set size of 200. . 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Decision Tree Classifier (DecisionTreeClassifier) **  

| Training Set Size | Training Time | Prediction Time (test) | F<sub>1</sub> Score (train) | F<sub>1</sub> Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0017 seconds        |   0.0005 seconds       |   1.0000         |   0.6942        |
| 200               |   0.0026 seconds        |   0.0003 seconds       |   1.0000         |   0.7597        |
| 300               |   0.0035 seconds        |   0.0004 seconds       |   1.0000         |   0.7154        |

** Support Vector Machine Classifier (SVC) **  

| Training Set Size | Training Time | Prediction Time (test) | F<sub>1</sub> Score (train) | F<sub>1</sub> Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0023 seconds        |   0.0016 seconds       |   0.8591         |   0.7838        |
| 200               |   0.0068 seconds        |   0.0033 seconds       |   0.8693         |   0.7755        |
| 300               |   0.0142 seconds        |   0.0036 seconds       |   0.8692         |   0.7586        |

** Ensamble Methods: Random Forest (RandomForestClassifier) **  

| Training Set Size | Training Time | Prediction Time (test) | F<sub>1</sub> Score (train) | F<sub>1</sub> Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0687 seconds        |   0.0100 seconds       |   1.0000         |   0.7597       |
| 200               |   0.0842 seconds        |   0.0103 seconds       |   0.9851         |   0.7704        |
| 300               |   0.0695 seconds        |   0.0104 seconds       |   0.9976         |   0.7970        |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **  From the Model Performance Matrix, it would appear that the **Ensamble Methods: Random Forest (RandomForestClassifier)** is the best model based on its final F<sub>1</sub> test scores of **0.7970**; however, we know that this learning model is based on the **Decision Tree Classifier**, which ran 20X faster in training and more than 25X faster predicting the test set.  Even though the F<sub>1</sub> test score for the **Decision Tree Classifier** seems low, we can see by the F<sub>1</sub> training score, that it is overfitting to the training data.  This overfitting can be tuned when we optimize the learning model in the next step.

Since we know we can achieve at least **0.7970** F<sub>1</sub> test score using **Random Forst**, which is based on the **Decision Tree Classifier**, against the same training and testing datasets, I assert we should be able to tune the **Decision Tree Classifier** to achieve the same **0.7970** or even better F<sub>1</sub> test score results.  Using **Occam's Razor Principle** (simpler is better), I assert that **Decision Tree classifier** is the best model out of the three selected model based on the available data, limited resources, cost and performance.


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Hello and thank you for allowing me to present the final model for the **Student Intervention** project.  This model is based on the **Decision Tree Classifier**, a learning model that generates *Decision Trees*, which uses a series of questions and intermidiate answers to obtain a final desired **prediction (passed|failed)**.  Below shows a prepresentation of what one of these questions and answer graphs or **trees** looks like when rendered:

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/CART_tree_titanic_survivors.png" title="A Decision Tree showing survival of passengers on the Titanic, CC BY-SA 3.0 by Stephen Milborrow">

The diagam above shows an example *Decision Tree*, which asks a series of questions and obtains a set of intermidiate answers to resolve a **prediction (survived|died)** for passwengers on the Titanic.  In this particular case, the question **is sex male?** is at the root of the *Decision Tree* and where all queries for predictions starts.  As can be seen in the diagram, your rate of survival is poor if you happen to be male and over 9.5 years old in this model.

**Model Training and Prediction Behavior:** In general, we will first split the student dataset into two: a training dataset and a testing dataset.  The training dataset will be given to the model to build the *Decision Tree* for our student intervention project, and the testing dataset will be use to score the accuracy of the generated *Decision Tree's* predictions.  The **Decision Tree Classifier** will analyze the student training dataset and determine the best question to ask near the root of the *Decision Tree* to have the best determination of which student in the dataset will pass or fail their graduation.  As additional questions and answers are derived and resolved, the **Decision Tree Classifier** will determine if it needs to ask another question to obtain a better prediction against the training set.  Once the **Decision Tree Classifier** has resolved all questions and generated a complete *Decision Tree*, predictions against new students can simply be obtained by applying the *Decision Tree* to the new students' information resulting in their pass or fail graduation predictions.  As can be seen, this is a simple model in both representation and execution while using our existing data, limited resources and budgets wisely.  We believe this new graduation prediction model will help the school district bring their graduation success rate up by identifying students that needs intervention early before graduation.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [203]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'criterion':('gini','entropy'), 'max_depth':[1,2,3], 'min_samples_split':[1,2,4,8,16,32,64,128], 'random_state':[0,1,3,5,7,11]}

# TODO: Initialize the classifier
clf = DecisionTreeClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train_200, y_train_200)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0003 seconds.
Tuned model has a training F1 score of 0.8136.
Made predictions in 0.0003 seconds.
Tuned model has a testing F1 score of 0.8029.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final F<sub>1</sub> test score for the tuned **Decision Tree Classifier** is **0.8029**.  This means we were able to achieve better performance than the **Ensamble Methods: Random Forest Classifier** by **0.0059!**, while at the same time reduce our prediction execution time by another 0.0001 to 0.0002 seconds.  Overall, the major tuning parameter that made the difference was *max_depth*.  This tuning parameter prevents the **Decision Tree Classifier** from overfitting to the training dataset to reduce bias to the training data and thus allow for variance with the testing data, and a better fit in general for the student population.

** Decision Tree Classifier (DecisionTreeClassifier):**  

| Model | Prediction Time (train) | Prediction Time (test) | F<sub>1</sub> Score (train) | F<sub>1</sub> Score (test) |
| :---------------: | :--------------------: | :--------------: | :-------------: |
| Untuned           |   0.0005 seconds        |   0.0004 seconds       |   1.0000         |   0.7154        |
| Tuned             |   0.0003 seconds        |   0.0003 seconds       |   0.8136         |   0.8029        |

The result from tuning the *max_depth* parameter reduced the overfitting to the training data from 1.000 to 0.8136 based on the F<sub>1</sub> training score, while bringing up the F<sub>1</sub> testing score from 0.7154 to 0.8029, meeting our objective of going above the **Random Forest Classifier** F<sub>1</sub> testing score of 0.7970.

In addition, we ran the tuned **Decision Tree Classifier** model through the *Confusion Matrix* to determine where the error was in the model.

** Confusion Matrix:**

| Scikit Learn Confusion Matrix Format |  | DecisionTreeClassifier Confusion Matrix |
| :----------: |  | :----------: |
|  ``TP   FN`` |  |  ``13   23`` |
|  ``FP   TN`` |  |  `` 4   55`` |


This means that we correctly predicted 68 passed|failed (13 *true positive* and 55 *true negatives*) out of the 95 students in the test dataset .  We have 4 *false positives* (students that failed graduation, but were predicted to pass) and 23 *false negatives* (students that successfully graduated, but were predicted to fail).  For our purpose, I believe that our bias to having more false negatives than false positives are acceptable because we would rather be more proactive in extending intervention to our students showing any signs of not being able to pass their graduations.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.